# Preprocessing

In [ ]:
conda activate qiime2-2021.4

In [2]:
mkdir 2021_BNW_skin
mkdir 16SV4_all
cd 16SV4_all

In [ ]:
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \ 
  --input-path /Users/christinanaesborg/Desktop/Raw_Samples/16SV4_BNW/Collected \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux.qza

In [ ]:
qiime demux summarize \
  --i-data demux.qza \
  --o-visualization demux.qzv

qiime tools view demux.qzv

In [ ]:
qiime metadata tabulate 
--m-input-file BNW-skin-stats-dada2.qza 
--o-visualization reports/BNW-skin-stats-dada2.qzv
qiime feature-table summarize 
--i-table BNW-skin-table.qza 
--o-visualization reports/BNW-skin-table.qzv
qiime feature-table tabulate-seqs 
--i-data BNW-skin-rep-seqs.qza 
--o-visualization reports/BNW-skin-rep-seqs.qzv

In [ ]:
# since sample id were only numbers, I had to reindex. Otherwise I would have incountered problems later on
qiime feature-table group \
  --i-table BNW-skin-table.qza \
  --p-axis sample \
  --m-metadata-file BNW-skin-metadata.tsv \
  --m-metadata-column SampleID-new \
  --p-mode sum \
  --o-grouped-table BNW-skin-reindexed-table.qza

In [ ]:
qiime feature-table summarize \
  --i-table BNW-skin-reindexed-table.qza \
  --m-sample-metadata-file BNW-skin-metadata.tsv \
  --o-visualization BNW-skin-reindexed-table.qzv

# Training SILVA database

In [ ]:
# import Qiime2 artifacts
qiime tools import \
  --type 'FeatureData[Sequence]' \
  --input-path /Users/christinanaesborg/Desktop/SILVA_132_QIIME_release/rep_set/rep_set_16S_only/99/silva_132_99_16S.fna  \
  --output-path silva_99_otus.qza

In [ ]:
qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-format HeaderlessTSVTaxonomyFormat \
  --input-path /Users/christinanaesborg/Desktop/SILVA_132_QIIME_release/taxonomy/16S_only/99/taxonomy_all_levels.txt  \        
  --output-path silva_99_ref-taxonomy.qza

In [ ]:
# extract reference reads
qiime feature-classifier extract-reads \
  --i-sequences silva_99_otus.qza \
  --p-f-primer GTGCCAGCMGCCGCGGTAA \
  --p-r-primer GGACTACHVGGGTWTCTAAT \
  --p-trunc-len 253  \
  --p-min-length 100 \
  --p-max-length 400 \
  --o-reads silva_ref-seqs.qza

In [ ]:
# train naive bayes classifier
qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads silva_ref-seqs.qza \
  --i-reference-taxonomy silva_99_ref-taxonomy.qza \
  --o-classifier silva_132_99_classifier.qza

In [ ]:
qiime feature-classifier classify-sklearn \
  --i-classifier silva_132_99_classifier.qza \
  --i-reads BNW-skin-rep-seqs.qza \
  --o-classification BNW-skin-silva-132.qza

In [ ]:
qiime metadata tabulate \
--m-input-file BNW-skin-silva-132.qza \
--o-visualization BNW-skin-silva-132.qzv

# Pre-trained SILVA database

In [ ]:
qiime feature-classifier classify-sklearn \
  --i-classifier silva-138-99-515-806-nb-classifier.qza \ 
  --i-reads BNW-skin-rep-seqs.qza \
  --o-classification BNW-skin-silva-138-pretrained-taxonomy.qza

In [ ]:
qiime metadata tabulate \
--m-input-file BNW-skin-silva-138-pretrained-taxonomy.qza \
--o-visualization BNW-skin-silva-138-pretrained-taxonomy.qzv

In [ ]:
# Create phylogenetic tree using sepp. Took a couple of days to run. 
#Output files: BNW-skin-sepp-tree.qza BNW-skin-sepp-placements.qza
qiime fragment-insertion sepp \
--i-representative-sequences BNW-skin-rep-seqs.qza \
--i-reference-database sepp-refs-silva-128.qza \
--o-tree BNW-skin-sepp-tree.qza \
--o-placements BNW-skin-sepp-placements.qza \
--verbose \
--p-threads 2

In [ ]:
qiime fragment-insertion filter-features \
  --i-table BNW-skin-table.qza \
  --i-tree BNW-skin-sepp-tree.qza \
  --o-filtered-table BNW-skin-filtered_table.qza \
  --o-removed-table BNW-skin-removed_table.qza \
  --verbose

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-removed_table.qza \
--o-visualization BNW-skin-removed_table.qzv

In [ ]:
Looks like nothing was removed, so will just use the unfiltered table (BNW-skin-table.qza) for subsequent analyses

In [ ]:
qiime diversity alpha-rarefaction \
--i-table BNW-skin-reindexed-table.qza \
--m-metadata-file /Users/christinanaesborg/Desktop/Microbiome/BNW-skin-metadata.tsv   \
--o-visualization BNW-skin-reindexed-table-rarefaction.qzv \
--p-min-depth 500 \
--p-max-depth 50000

# output: BNW-skin-reindexed-table-rarefaction.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-reindexed-table.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-min-frequency 12201 \
--o-filtered-table BNW-table-atleast-12201-reads.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-table-atleast-12201-reads.qza \
--o-visualization BNW-table-atleast-12201-reads.qzv

# Remove toe samples 

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-reindexed-table.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "bodysite IN ('toe')" \
--p-exclude-ids \
--o-filtered-table BNW-skin-table-no-toe.qza

In [ ]:
# remove samples with low reads
qiime feature-table filter-samples \
--i-table BNW-skin-table-no-toe.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-min-frequency 12201 \
--o-filtered-table BNW-skin-table-no-toe-at-least-12201-reads.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-no-toe-at-least-12201-reads.qza \
--o-visualization BNW-skin-table-no-toe-at-least-12201-reads.qzv 

In [ ]:
# create taxa barplot per individual sample
qiime taxa barplot \
--i-table BNW-skin-table-no-toe-at-least-12201-reads.qza \
--i-taxonomy BNW-skin-silva-138-pretrained-taxonomy.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--o-visualization BNW-skin-table-no-toe-at-least-12201-reads-BARPLOT-PER-SAMPLE.qzv
# w33 has a larger relative frequency of cyanobacteria than the others, but I will keep it for now.

In [ ]:
# collapse samples by severity and make barplots
qiime feature-table group \
--i-table BNW-skin-table-no-toe-at-least-12201-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column Severity \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-skin-table-no-toe-SEVERITY.qza

In [ ]:
qiime taxa barplot \
--i-table BNW-skin-table-no-toe-SEVERITY.qza \
--i-taxonomy BNW-skin-silva-138-pretrained-taxonomy.qza \
--m-metadata-file BNW-skin-metadata-SEVERITY.tsv \
--o-visualization BNW-skin-table-no-toe-SEVERITY-BARPLOT.qzv

In [ ]:
# core diversity on filtered table using 13084 rarefaction depth
qiime diversity core-metrics-phylogenetic \
--i-table BNW-skin-table-no-toe-at-least-12201-reads.qza \
--i-phylogeny BNW-skin-sepp-tree.qza \
--p-sampling-depth 13084 \
--m-metadata-file BNW-skin-metadata.tsv \
--output-dir BNW-skin-Core-metrics-final-filtered-table-13084

In [ ]:
for i in BNW-skin-Core-metrics-final-filtered-table-13084/*vector.qza; do qiime diversity alpha-group-significance --i-alpha-diversity $i --m-metadata-file BNW-skin-metadata.tsv --o-visualization ${i/.qza/.qzv}; done

# Permanova test for microbial composition differences

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix BNW-skin-Core-metrics-final-filtered-table-13084/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column samplegrouping \
--o-visualization BNW-skin-Core-metrics-final-filtered-table-13084/unweighted_unifrac_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix BNW-skin-Core-metrics-final-filtered-table-13084/weighted_unifrac_distance_matrix.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column samplegrouping \
--o-visualization BNW-skin-Core-metrics-final-filtered-table-13084/weighted_unifrac_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

# ANCOM analysis to identify differentially abundant features/OTUS

In [ ]:
#remove low abundance/prevalence features
qiime feature-table filter-features \
--i-table BNW-skin-table-no-toe-at-least-12201-reads.qza \
--o-filtered-table BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500.qza \
--p-min-samples 8 \
--p-min-frequency 500

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500.qza \
--o-visualization BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500.qzv

In [ ]:
qiime composition add-pseudocount \
--i-table BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500.qza \
--o-composition-table BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500-pseudo.qza

In [ ]:
qiime composition ancom \
--i-table BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500-pseudo.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column samplegrouping \
--o-visualization BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500-pseudo-SEVERITY.qzv

# ANCOM collapsed at family level (i.e. not at the feature level)

In [ ]:
qiime taxa collapse \
--i-table BNW-skin-table-no-toe-at-least-12201-reads.qza  \
--i-taxonomy BNW-skin-silva-138-pretrained-taxonomy.qza \
--p-level 5 \
--o-collapsed-table BNW-skin-table-no-toe-FAMILY.qza

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-no-toe-FAMILY.qza \
--o-filtered-table BNW-skin-table-no-toe-FAMILY-FILTERED-ANCOM-ms8-mf500.qza \
--p-min-samples 8 \
--p-min-frequency 500

In [ ]:
qiime composition add-pseudocount \
--i-table BNW-skin-table-no-toe-FAMILY-FILTERED-ANCOM-ms8-mf500.qza  \
--o-composition-table BNW-skin-table-no-toe-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo.qza 

In [ ]:
qiime composition ancom \
--i-table BNW-skin-table-no-toe-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column Severity \
--o-visualization BNW-skin-table-no-toe-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo-SEVERITY.qzv

# Unique feature analysis

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table-no-toe-at-least-12201-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "[Severity]='Confidently healthy'" \
--o-filtered-table BNW-skin-table-no-toe-CONFHEALTHYonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-no-toe-CONFHEALTHYonly.qza \
--o-visualization BNW-skin-table-no-toe-CONFHEALTHYonly.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table-no-toe-at-least-12201-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "[Severity]='Observational healthy'" \
--o-filtered-table BNW-skin-table-no-toe-OBSHEALTHYonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-no-toe-OBSHEALTHYonly.qza \
--o-visualization BNW-skin-table-no-toe-OBSHEALTHYonly.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table-no-toe-at-least-12201-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "[Severity]='Uncertain/low mange'" \
--o-filtered-table BNW-skin-table-no-toe-UNCERTAINonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-no-toe-UNCERTAINonly.qza \
--o-visualization BNW-skin-table-no-toe-UNCERTAINonly.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table-no-toe-at-least-12201-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "[Severity]='Mangy'" \
--o-filtered-table BNW-skin-table-no-toe-MANGYonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-no-toe-MANGYonly.qza \
--o-visualization BNW-skin-table-no-toe-MANGYonly.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table-no-toe-at-least-12201-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "[Severity]='Severe mange'" \
--o-filtered-table BNW-skin-table-no-toe-SEVEREonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-no-toe-SEVEREonly.qza \
--o-visualization BNW-skin-table-no-toe-SEVEREonly.qzv

# Provide overview of ASVs for each severity type

In [ ]:
qiime tools export \
--input-path BNW-skin-table-no-toe-CONFHEALTHYonly.qza \
--output-path table-no-toe-CONFHEALTHYonly.biom

In [ ]:
biom convert -i table-no-toe-CONFHEALTHYonly.biom/feature-table.biom -o table-no-toe-CONFHEALTHYonly.biom/feature-table.tsv --to-tsv

In [ ]:
qiime tools export \
--input-path BNW-skin-table-no-toe-OBSHEALTHYonly.qza \
--output-path table-no-toe-OBSHEALTHYonly.biom

In [ ]:
biom convert -i table-no-toe-OBSHEALTHYonly.biom/feature-table.biom -o table-no-toe-OBSHEALTHYonly.biom/feature-table.tsv --to-tsv

In [ ]:
qiime tools export \
--input-path BNW-skin-table-no-toe-UNCERTAINonly.qza \
--output-path table-no-toe-UNCERTAINonly.biom

In [ ]:
biom convert -i table-no-toe-UNCERTAINonly.biom/feature-table.biom -o table-no-toe-UNCERTAINonly.biom/feature-table.tsv --to-tsv

In [ ]:
qiime tools export \
--input-path BNW-skin-table-no-toe-MANGYonly.qza \
--output-path table-no-toe-MANGYonly.biom

In [ ]:
biom convert -i table-no-toe-MANGYonly.biom/feature-table.biom -o table-no-toe-MANGYonly.biom/feature-table.tsv --to-tsv

In [ ]:
qiime tools export \
--input-path BNW-skin-table-no-toe-SEVEREonly.qza \
--output-path table-no-toe-SEVEREonly.biom

In [ ]:
biom convert -i table-no-toe-SEVEREonly.biom/feature-table.biom -o table-no-toe-SEVEREonly.biom/feature-table.tsv --to-tsv

# Unique ASVs - min 5 samples due to severity groupings

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-no-toe-CONFHEALTHYonly.qza \
--o-filtered-table BNW-skin-table-no-toe-CONFHEALTHYonly-ms5-mf142.qza \
--p-min-samples 5 \
--p-min-frequency 142

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-no-toe-CONFHEALTHYonly-ms5-mf142.qza \
--o-visualization BNW-skin-table-no-toe-CONFHEALTHYonly-ms5-mf142.qzv

Pre-filter
Number of features: 2807
Total frequency: 950,162

Post-filter
Number of features: 139
Total frequency: 627,314

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-no-toe-OBSHEALTHYonly.qza \
--o-filtered-table BNW-skin-table-no-toe-OBSHEALTHYonly-ms5-mf45.qza \
--p-min-samples 5 \
--p-min-frequency 145

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-no-toe-OBSHEALTHYonly-ms5-mf45.qza \
--o-visualization BNW-skin-table-no-toe-OBSHEALTHYonly-ms5-mf45.qzv

Pre-filter
Number of features: 1249
Total frequency: 304,193

Post-filter
Number of features: 16
Total frequency: 122,210

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-no-toe-UNCERTAINonly.qza \
--o-filtered-table BNW-skin-table-no-toe-UNCERTAINonly-ms5-mf276.qza \
--p-min-samples 5 \
--p-min-frequency 276

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-no-toe-UNCERTAINonly-ms5-mf276.qza \
--o-visualization BNW-skin-table-no-toe-UNCERTAINonly-ms5-mf276.qzv

Pre-filter
Number of features: 4699
Total frequency: 1,845,342

Post-filter
Number of features: 257
Total frequency: 1,477,965

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-no-toe-MANGYonly.qza \
--o-filtered-table BNW-skin-table-no-toe-MANGYonly-ms5-mf56.qza \
--p-min-samples 5 \
--p-min-frequency 56

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-no-toe-MANGYonly-ms5-mf56.qza \
--o-visualization BNW-skin-table-no-toe-MANGYonly-ms5-mf56.qzv

Pre-filter
Number of features: 870
Total frequency: 374,474

Post-filter
Number of features: 15
Total frequency: 76,575

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-no-toe-SEVEREonly.qza \
--o-filtered-table BNW-skin-table-no-toe-SEVEREonly-ms5-mf109.qza \
--p-min-samples 5 \
--p-min-frequency 109

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-no-toe-SEVEREonly-ms5-mf109.qza \
--o-visualization BNW-skin-table-no-toe-SEVEREonly-ms5-mf109.qzv

Pre-filter
Number of features: 481
Total frequency: 730,694

Post-filter
Number of features: 37
Total frequency: 392,605

# Remove toe samples and groups of Uncertain/low mange and Observational healthy from analysis

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-reindexed-table.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "[bodysite] IN ('toe') AND [Severity] IN ('Uncertain/low mange','Observational healthy')" \
--p-exclude-ids \
--o-filtered-table FinalCat/BNW-skin-table-no-toe_FinalCat.qza

In [ ]:
# remove samples with low reads
qiime feature-table filter-samples \
--i-table FinalCat/BNW-skin-table-no-toe_FinalCat.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-min-frequency 12201 \
--o-filtered-table FinalCat/BNW-skin-table-no-toe-at-least-12201-reads_FinalCat.qza

In [ ]:
qiime feature-table summarize \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-12201-reads_FinalCat.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-at-least-12201-reads_FinalCat.qzv 

In [ ]:
# create taxa barplot per individual sample
qiime taxa barplot \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-12201-reads_FinalCat.qza \
--i-taxonomy BNW-skin-silva-138-pretrained-taxonomy.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--o-visualization FinalCat/BNW-skin-table-no-toe-at-least-12201-reads-BARPLOT-PER-SAMPLE.qzv
# w33 has a larger relative frequency of cyanobacteria than the others, but I will keep it for now.

In [ ]:
# collapse samples by severity and make barplots
qiime feature-table group \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-12201-reads_FinalCat.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column Severity \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table FinalCat/BNW-skin-table-no-toe-SEVERITY_FinalCat.qza

In [ ]:
qiime taxa barplot \
--i-table FinalCat/BNW-skin-table-no-toe-SEVERITY_FinalCat.qza \
--i-taxonomy BNW-skin-silva-138-pretrained-taxonomy.qza \
--m-metadata-file BNW-skin-metadata-SEVERITY.tsv \
--o-visualization FinalCat/BNW-skin-table-no-toe-SEVERITY-BARPLOT.qzv

In [ ]:
# core diversity on filtered table using 13084 rarefaction depth
qiime diversity core-metrics-phylogenetic \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-12201-reads_FinalCat.qza \
--i-phylogeny BNW-skin-sepp-tree.qza \
--p-sampling-depth 13084 \
--m-metadata-file BNW-skin-metadata.tsv \
--output-dir FinalCat-BNW-skin-Core-metrics-final-filtered-table-13084

In [ ]:
for i in FinalCat-BNW-skin-Core-metrics-final-filtered-table-13084/*vector.qza; do qiime diversity alpha-group-significance --i-alpha-diversity $i --m-metadata-file BNW-skin-metadata.tsv --o-visualization ${i/.qza/.qzv}; done

In [ ]:
qiime diversity alpha \
  --i-table FinalCat/BNW-skin-table-no-toe-at-least-12201-reads_FinalCat.qza \
  --p-metric chao1 \
  --o-alpha-diversity FinalCat-BNW-skin-Core-metrics-final-filtered-table-13084/chao1_index.qza

In [ ]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity FinalCat-BNW-skin-Core-metrics-final-filtered-table-13084/chao1_index.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --o-visualization FinalCat-BNW-skin-Core-metrics-final-filtered-table-13084/chao1_index.qzv

# Permanova test for microbial composition differences

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix FinalCat-BNW-skin-Core-metrics-final-filtered-table-13084/bray_curtis_distance_matrix.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column Severity \
--o-visualization FinalCat-BNW-skin-Core-metrics-final-filtered-table-13084/bray_curtis_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix FinalCat-BNW-skin-Core-metrics-final-filtered-table-13084/jaccard_distance_matrix.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column Severity \
--o-visualization FinalCat-BNW-skin-Core-metrics-final-filtered-table-13084/jaccard_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

# ANCOM analysis to identify differentially abundant features/OTUS

In [ ]:
#remove low abundance/prevalence features
qiime feature-table filter-features \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-12201-reads_FinalCat.qza \
--o-filtered-table FinalCat/BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500_FinalCat.qza \
--p-min-samples 8 \
--p-min-frequency 500

In [ ]:
qiime feature-table summarize \
--i-table FinalCat/BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500_FinalCat.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500_FinalCat.qzv

In [ ]:
qiime composition add-pseudocount \
--i-table FinalCat/BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500_FinalCat.qza \
--o-composition-table FinalCat/BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500-pseudo_FinalCat.qza

In [ ]:
qiime composition ancom \
--i-table FinalCat/BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500-pseudo_FinalCat.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column Severity \
--o-visualization FinalCat/BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500-pseudo-SEVERITY_FinalCat.qzv

# ANCOM collapsed at family level (i.e. not at the feature level)

In [ ]:
qiime taxa collapse \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-12201-reads_FinalCat.qza  \
--i-taxonomy BNW-skin-silva-138-pretrained-taxonomy.qza \
--p-level 5 \
--o-collapsed-table FinalCat/BNW-skin-table-no-toe-FAMILY_FinalCat.qza

In [ ]:
qiime feature-table filter-features \
--i-table FinalCat/BNW-skin-table-no-toe-FAMILY_FinalCat.qza \
--o-filtered-table FinalCat/BNW-skin-table-no-toe-FAMILY-FILTERED-ANCOM-ms8-mf500_FinalCat.qza \
--p-min-samples 8 \
--p-min-frequency 500

In [ ]:
qiime composition add-pseudocount \
--i-table FinalCat/BNW-skin-table-no-toe-FAMILY-FILTERED-ANCOM-ms8-mf500_FinalCat.qza  \
--o-composition-table FinalCat/BNW-skin-table-no-toe-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo_FinalCat.qza 

In [ ]:
qiime composition ancom \
--i-table FinalCat/BNW-skin-table-no-toe-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo_FinalCat.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column Severity \
--o-visualization FinalCat/BNW-skin-table-no-toe-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo-SEVERITY_FinalCat.qzv

# Unique feature analysis

In [ ]:
qiime feature-table filter-samples \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-12201-reads_FinalCat.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "[Severity]='Confidently healthy'" \
--o-filtered-table FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly.qza

In [ ]:
qiime feature-table summarize \
--i-table FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-12201-reads_FinalCat.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "[Severity]='Mangy'" \
--o-filtered-table FinalCat/BNW-skin-table-no-toe-MANGYonly.qza

In [ ]:
qiime feature-table summarize \
--i-table FinalCat/BNW-skin-table-no-toe-MANGYonly.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-MANGYonly.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-12201-reads_FinalCat.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "[Severity]='Severe mange'" \
--o-filtered-table FinalCat/BNW-skin-table-no-toe-SEVEREonly.qza

In [ ]:
qiime feature-table summarize \
--i-table FinalCat/BNW-skin-table-no-toe-SEVEREonly.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-SEVEREonly.qzv

# Provide overview of ASVs for each severity type

In [ ]:
qiime tools export \
--input-path FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly.qza \
--output-path FinalCat-table-no-toe-CONFHEALTHYonly.biom

In [ ]:
biom convert -i FinalCat-table-no-toe-CONFHEALTHYonly.biom/feature-table.biom -o FinalCat-table-no-toe-CONFHEALTHYonly.biom/feature-table.tsv --to-tsv

In [ ]:
qiime tools export \
--input-path FinalCat/BNW-skin-table-no-toe-MANGYonly.qza \
--output-path FinalCat-table-no-toe-MANGYonly.biom

In [ ]:
biom convert -i FinalCat-table-no-toe-MANGYonly.biom/feature-table.biom -o FinalCat-table-no-toe-MANGYonly.biom/feature-table.tsv --to-tsv

In [ ]:
qiime tools export \
--input-path FinalCat/BNW-skin-table-no-toe-SEVEREonly.qza \
--output-path FinalCat-table-no-toe-SEVEREonly.biom

In [ ]:
biom convert -i FinalCat-table-no-toe-SEVEREonly.biom/feature-table.biom -o FinalCat-table-no-toe-SEVEREonly.biom/feature-table.tsv --to-tsv

# Unique ASVs - min 3 samples due to severity groupings

In [ ]:
qiime feature-table filter-features \
--i-table FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly.qza \
--o-filtered-table FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly-ms3-mf142.qza \
--p-min-samples 3 \
--p-min-frequency 142

In [ ]:
qiime feature-table summarize \
--i-table FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly-ms3-mf142.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly-ms3-mf142.qzv

Pre-filter
Number of features: 2807
Total frequency: 950,162

Post-filter
Number of features: 262
Total frequency: 778,217

In [ ]:
# provide overview of unique ASVs
qiime tools export \
--input-path FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly-ms3-mf142.qza \
--output-path FinalCat/FinalCat-table-no-toe-CONFHEALTHYonly-ms3-mf142.biom

In [ ]:
#convert biom to tsv file
biom convert -i FinalCat/FinalCat-table-no-toe-CONFHEALTHYonly-ms3-mf142.biom/feature-table.biom -o FinalCat/FinalCat-table-no-toe-CONFHEALTHYonly-ms3-mf142.biom/feature-table.tsv --to-tsv

In [ ]:
qiime feature-table filter-features \
--i-table FinalCat/BNW-skin-table-no-toe-MANGYonly.qza \
--o-filtered-table FinalCat/BNW-skin-table-no-toe-MANGYonly-ms3-mf56.qza \
--p-min-samples 3 \
--p-min-frequency 56

In [ ]:
qiime feature-table summarize \
--i-table FinalCat/BNW-skin-table-no-toe-MANGYonly-ms3-mf56.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-MANGYonly-ms3-mf56.qzv

Pre-filter
Number of features: 870
Total frequency: 374,474

Post-filter
Number of features: 95
Total frequency: 237,244

In [ ]:
# provide overview of unique ASVs
qiime tools export \
--input-path FinalCat/BNW-skin-table-no-toe-MANGYonly-ms3-mf56.qza \
--output-path FinalCat/FinalCat-table-no-toe-MANGYonly-ms3-mf56.biom

In [ ]:
#convert biom to tsv file
biom convert -i FinalCat/FinalCat-table-no-toe-MANGYonly-ms3-mf56.biom/feature-table.biom -o FinalCat/FinalCat-table-no-toe-MANGYonly-ms3-mf56.biom/feature-table.tsv --to-tsv

In [ ]:
qiime feature-table filter-features \
--i-table FinalCat/BNW-skin-table-no-toe-SEVEREonly.qza \
--o-filtered-table FinalCat/BNW-skin-table-no-toe-SEVEREonly-ms3-mf109.qza \
--p-min-samples 3 \
--p-min-frequency 109

In [ ]:
qiime feature-table summarize \
--i-table FinalCat/BNW-skin-table-no-toe-SEVEREonly-ms3-mf109.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-SEVEREonly-ms3-mf109.qzv

In [ ]:
# provide overview of unique ASVs
qiime tools export \
--input-path FinalCat/BNW-skin-table-no-toe-SEVEREonly-ms3-mf109.qza \
--output-path FinalCat/FinalCat-table-no-toe-SEVEREonly-ms3-mf109.biom

In [ ]:
#convert biom to tsv file
biom convert -i FinalCat/FinalCat-table-no-toe-SEVEREonly-ms3-mf109.biom/feature-table.biom -o FinalCat/FinalCat-table-no-toe-SEVEREonly-ms3-mf109.biom/feature-table.tsv --to-tsv

# Make taxa barplots

In [ ]:
qiime feature-table group \
--i-table FinalCat/BNW-skin-table-no-toe-SEVEREonly-ms3-mf109.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column wombatid \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table FinalCat/BNW-skin-table-no-toe-SEVERE-ms3-mf109-wombat-COLLAPSED.qza

In [ ]:
qiime feature-table group \
--i-table FinalCat/BNW-skin-table-no-toe-MANGYonly-ms3-mf56.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column wombatid \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table FinalCat/BNW-skin-table-no-toe-MANGY-ms3-mf56-wombat-COLLAPSED.qza

In [ ]:
qiime feature-table group \
--i-table FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly-ms3-mf142.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column wombatid \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table FinalCat/BNW-skin-table-no-toe-CONFHEALTHY-ms3-mf142-wombat-COLLAPSED.qza

In [ ]:
qiime feature-table merge \
--i-tables FinalCat/BNW-skin-table-no-toe-CONFHEALTHY-ms3-mf142-wombat-COLLAPSED.qza FinalCat/BNW-skin-table-no-toe-MANGY-ms3-mf56-wombat-COLLAPSED.qza FinalCat/BNW-skin-table-no-toe-SEVERE-ms3-mf109-wombat-COLLAPSED.qza \
--o-merged-table FinalCat/BNW-skin-table-no-toe-wombat-COLLAPSED.qza

In [ ]:
qiime taxa barplot \
--i-table FinalCat/BNW-skin-table-no-toe-wombat-COLLAPSED.qza \
--i-taxonomy BNW-skin-silva-138-pretrained-taxonomy.qza \
--m-metadata-file BNW-skin-metadata-WOM.tsv \
--o-visualization FinalCat/BNW-skin-table-no-toe-WOMBAT-COLLAPSED-BARPLOT.qzv

In [ ]:
qiime feature-table group \
--i-table FinalCat/BNW-skin-table-no-toe-SEVEREonly-ms3-mf109.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column Severity \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table FinalCat/BNW-skin-table-no-toe-SEVERE-ms3-mf109-COLLAPSED.qza

In [ ]:
qiime feature-table group \
--i-table FinalCat/BNW-skin-table-no-toe-MANGYonly-ms3-mf56.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column Severity \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table FinalCat/BNW-skin-table-no-toe-MANGY-ms3-mf56-COLLAPSED.qza

In [ ]:
qiime feature-table group \
--i-table FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly-ms3-mf142.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column Severity \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table FinalCat/BNW-skin-table-no-toe-CONFHEALTHY-ms3-mf142-COLLAPSED.qza

In [ ]:
qiime feature-table merge \
--i-tables FinalCat/BNW-skin-table-no-toe-CONFHEALTHY-ms3-mf142-COLLAPSED.qza FinalCat/BNW-skin-table-no-toe-MANGY-ms3-mf56-COLLAPSED.qza FinalCat/BNW-skin-table-no-toe-SEVERE-ms3-mf109-COLLAPSED.qza \
--o-merged-table FinalCat/BNW-skin-table-no-toe-severity-COLLAPSED.qza

In [ ]:
qiime taxa barplot \
--i-table FinalCat/BNW-skin-table-no-toe-severity-COLLAPSED.qza \
--i-taxonomy BNW-skin-silva-138-pretrained-taxonomy.qza \
--m-metadata-file BNW-skin-metadata-SEVERITY.tsv \
--o-visualization FinalCat/BNW-skin-table-no-toe-SEVERITY-COLLAPSED-BARPLOT.qzv

# Core metrics analysis

In [ ]:
qiime feature-table core-features \
--i-table FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly-ms3-mf142.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly-ms3-mf142-CORE-FEATURES.qzv

In [ ]:
qiime feature-table core-features \
--i-table FinalCat/BNW-skin-table-no-toe-MANGYonly-ms3-mf56.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-MANGYonly-ms3-mf56-CORE-FEATURES.qzv

In [ ]:
qiime feature-table core-features \
--i-table FinalCat/BNW-skin-table-no-toe-SEVEREonly-ms3-mf109.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-SEVEREonly-ms3-mf109-CORE-FEATURES.qzv

Merge filtered tables to look for core microbiome accross all wombats (i.e. healthy, mangy, severe)

In [ ]:
qiime feature-table merge \
--i-tables FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly-ms3-mf142.qza FinalCat/BNW-skin-table-no-toe-MANGYonly-ms3-mf56.qza FinalCat/BNW-skin-table-no-toe-SEVEREonly-ms3-mf109.qza \
--o-merged-table FinalCat/BNW-skin-table-no-toe-FILTERED-MERGED.qza

In [ ]:
qiime feature-table core-features \
--i-table FinalCat/BNW-skin-table-no-toe-FILTERED-MERGED.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-FILTERED-MERGED-CORE-FEATURES.qzv

# Pull out a list of features from each of the core microbiota tables to filter the main tables from (to work out abundance of the core microbiota):

In [ ]:
for i in *.tsv; do awk '{print $1}' $i > ${i/.tsv/-FEATURES.tsv}; done

In [ ]:
for i in *.tsv; do sed -i'' 's/Feature/feature id/g' $i; done

# Taxa barplots

In [ ]:
qiime taxa barplot \
--i-table BNW-table-atleast-12201-reads.qza \
--i-taxonomy BNW-skin-silva-138-pretrained-taxonomy.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--o-visualization BNW-table-atleast-12201-reads-BARPLOT-PER-SAMPLE.qzv

In [ ]:
qiime feature-table group \
--i-table BNW-table-atleast-12201-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column infectionstatus \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-table-atleast-12201-reads-INFECTION-STATUS.qza

In [ ]:
qiime taxa barplot \
--i-table BNW-table-atleast-12201-reads-INFECTION-STATUS.qza \
--i-taxonomy BNW-skin-silva-138-pretrained-taxonomy.qza \
--m-metadata-file BNW-skin-metadata-infection.tsv \
--o-visualization BNW-table-atleast-12201-reads-INFECTION-STATUS-BARPLOT.qzv

In [ ]:
qiime feature-table group \
--i-table BNW-table-atleast-12201-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column samplegrouping \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-table-atleast-12201-reads-SAMPLE-GROUPINGS.qza

In [ ]:
qiime taxa barplot \
--i-table BNW-table-atleast-12201-reads-SAMPLE-GROUPINGS.qza \
--i-taxonomy BNW-skin-silva-138-pretrained-taxonomy.qza \
--m-metadata-file BNW-skin-metadata-GROUPINGS.tsv \
--o-visualization BNW-table-atleast-12201-reads-SAMPLE-GROUPINGS-BARPLOT.qzv

In [ ]:
qiime feature-table group \
--i-table BNW-table-atleast-12201-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column wombatid \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-table-atleast-12201-reads-WOMBAT.qza

In [ ]:
qiime taxa barplot \
--i-table BNW-table-atleast-12201-reads-WOMBAT.qza \
--i-taxonomy BNW-skin-silva-138-pretrained-taxonomy.qza \
--m-metadata-file BNW-skin-metadata-WOM.tsv \
--o-visualization BNW-table-atleast-12201-reads-WOMBAT-BARPLOT.qzv

In [ ]:
qiime feature-table group \
--i-table BNW-table-atleast-12201-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column Severity \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-table-atleast-12201-reads-SEVERITY.qza

In [ ]:
qiime taxa barplot \
--i-table BNW-table-atleast-12201-reads-SEVERITY.qza \
--i-taxonomy BNW-skin-silva-138-pretrained-taxonomy.qza \
--m-metadata-file BNW-skin-metadata-SEVERITY.tsv \
--o-visualization BNW-table-atleast-12201-reads-SEVERITY-BARPLOT.qzv

# Alpha/Beta diversity

In [ ]:
qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences BNW-skin-rep-seqs.qza \
  --o-alignment silva-aligned-rep-seqs.qza \
  --o-masked-alignment silva-masked-aligned-rep-seqs.qza \
  --o-tree silva-unrooted-tree.qza \
  --o-rooted-tree silva-rooted-tree.qza

In [ ]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny silva-rooted-tree.qza \
  --i-table BNW-skin-reindexed-table.qza \
  --p-sampling-depth 12201 \
  --m-metadata-file BNW-skin-metadata.tsv  \
  --output-dir silva-core-metrics

In [ ]:
# alpha diversity
qiime diversity alpha-group-significance \
  --i-alpha-diversity silva-core-metrics/faith_pd_vector.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --o-visualization silva-core-metrics/faith-pd-group-significance.qzv

In [ ]:
# alpha diversity
qiime diversity alpha-group-significance \
  --i-alpha-diversity silva-core-metrics/evenness_vector.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --o-visualization silva-core-metrics/evenness-group-significance.qzv

In [ ]:
# beta diversity - between body sites
qiime diversity beta-group-significance \
  --i-distance-matrix silva-core-metrics/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --m-metadata-column bodysite \
  --o-visualization silva-core-metrics/unweighted-unifrac-body-site-significance.qzv \
  --p-pairwise

In [ ]:
# beta diversity - between wombats
qiime diversity beta-group-significance \
  --i-distance-matrix silva-core-metrics/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --m-metadata-column wombatid \
  --o-visualization silva-core-metrics/unweighted-unifrac-wombat-significance.qzv \
  --p-pairwise

In [ ]:
# beta diversity - infection status
qiime diversity beta-group-significance \
  --i-distance-matrix silva-core-metrics/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --m-metadata-column infectionstatus \
  --o-visualization silva-core-metrics/unweighted-unifrac-infection-status-significance.qzv \
  --p-pairwise

In [ ]:
qiime metadata tabulate \
  --m-input-file BNW-skin-metadata.tsv \
  --m-input-file faith_pd_vector.qza observed_features_vector.qza \
  --o-visualization reindexed-tabulated-combined-metadata.qzv

# Differential abundance testing with ANCOM (family level)

In [ ]:
qiime taxa collapse \
--i-table BNW-table-atleast-12201-reads.qza \
--i-taxonomy BNW-skin-silva-138-pretrained-taxonomy.qza \
--p-level 5 \
--o-collapsed-table BNW-skin-table-final-FAMILY.qza

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-final-FAMILY.qza \
--o-filtered-table BNW-skin-table-final-FAMILY-FILTERED-ANCOM-ms8-mf500.qza \
--p-min-samples 8 \
--p-min-frequency 500

In [ ]:
qiime composition add-pseudocount \
--i-table BNW-skin-table-final-FAMILY-FILTERED-ANCOM-ms8-mf500.qza \
--o-composition-table BNW-skin-table-final-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo.qza

In [ ]:
qiime composition ancom \
--i-table BNW-skin-table-final-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column Severity \
--o-visualization BNW-skin-table-final-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo-SEVERITY.qzv

# Unique feature analysis 

In [ ]:
#seperate main table to only include observational healthy samples for wombats
qiime feature-table filter-samples \
--i-table BNW-table-atleast-12201-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "[Severity]='Observational healthy'" \
--o-filtered-table BNW-skin-table-final-OBSHEALTHYonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-final-OBSHEALTHYonly.qza \
--o-visualization BNW-skin-table-final-OBSHEALTHYonly.qzv

In [ ]:
#seperate main table to only include confidently healthy samples for wombats
qiime feature-table filter-samples \
--i-table BNW-table-atleast-12201-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "[Severity]='Confidently healthy'" \
--o-filtered-table BNW-skin-table-final-CONFHEALTHYonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-final-CONFHEALTHYonly.qza \
--o-visualization BNW-skin-table-final-CONFHEALTHYonly.qzv

In [ ]:
#seperate main table to only include uncertain/low mange samples for wombats
qiime feature-table filter-samples \
--i-table BNW-table-atleast-12201-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "[Severity]='Uncertain/low mange'" \
--o-filtered-table BNW-skin-table-final-UNCERTAINonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-final-UNCERTAINonly.qza \
--o-visualization BNW-skin-table-final-UNCERTAINonly.qzv

In [ ]:
#seperate main table to only include severe mangy samples for wombats
qiime feature-table filter-samples \
--i-table BNW-table-atleast-12201-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "[Severity]='Severe mange'" \
--o-filtered-table BNW-skin-table-final-SEVEREonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-final-SEVEREonly.qza \
--o-visualization BNW-skin-table-final-SEVEREonly.qzv

In [ ]:
#seperate main table to only include severe mangy samples for wombats
qiime feature-table filter-samples \
--i-table BNW-table-atleast-12201-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "[Severity]='Mangy'" \
--o-filtered-table BNW-skin-table-final-MANGYonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-final-MANGYonly.qza \
--o-visualization BNW-skin-table-final-MANGYonly.qzv

# Export to biom file - done for all of the unique feature data analysis qza-files

qiime tools export \
--input-path BNW-skin-table-final-OBSHEALTHYonly.qza \
--output-path BNW-skin-table-final-OBSHEALTHYonly.biom

In [ ]:
biom convert -i BNW-skin-table-final-OBSHEALTHYonly.biom/feature-table.biom -o BNW-skin-table-final-OBSHEALTHYonly.biom/feature-table.tsv --to-tsv

# Core metrics analysis

In [ ]:
qiime feature-table core-features \
--i-table BNW-skin-table-final-OBSHEALTHYonly.qza \
--o-visualization BNW-skin-table-final-OBSHEALTHYonly-CORE-FEATURES.qzv

In [ ]:
qiime feature-table core-features \
--i-table BNW-skin-table-final-CONFHEALTHYonly.qza \
--o-visualization BNW-skin-table-final-CONFHEALTHYonly-CORE-FEATURES.qzv

In [ ]:
qiime feature-table core-features \
--i-table BNW-skin-table-final-UNCERTAINonly.qza \
--o-visualization BNW-skin-table-final-UNCERTAINonly-CORE-FEATURES.qzv

In [ ]:
qiime feature-table core-features \
--i-table BNW-skin-table-final-SEVEREonly.qza \
--o-visualization BNW-skin-table-final-SEVEREonly-CORE-FEATURES.qzv

In [ ]:
qiime feature-table core-features \
--i-table BNW-skin-table-final-MANGYonly.qza \
--o-visualization BNW-skin-table-final-MANGYEonly-CORE-FEATURES.qzv

# filter features from each table with a per-table frequency of < 0.00015%, and fewer than 2 samples.

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-final-OBSHEALTHYonly.qza \
--o-filtered-table BNW-skin-table-final-OBSHEALTHY-ms2-mf67.qza \
--p-min-samples 2 \
--p-min-frequency 67

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-final-OBSHEALTHY-ms2-mf67.qza \
--o-visualization BNW-skin-table-final-OBSHEALTHY-ms2-mf67.qzv

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-final-CONFHEALTHYonly.qza \
--o-filtered-table BNW-skin-table-final-CONFHEALTHY-ms2-mf198.qza \
--p-min-samples 2 \
--p-min-frequency 198

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-final-CONFHEALTHY-ms2-mf198.qza \
--o-visualization BNW-skin-table-final-CONFHEALTHY-ms2-mf198.qzv

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-final-UNCERTAINonly.qza \
--o-filtered-table BNW-skin-table-final-UNCERTAIN-ms2-mf389.qza \
--p-min-samples 2 \
--p-min-frequency 389

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-final-UNCERTAIN-ms2-mf389.qza \
--o-visualization BNW-skin-table-final-UNCERTAIN-ms2-mf389.qzv

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-final-MANGYonly.qza \
--o-filtered-table BNW-skin-table-final-MANGY-ms2-mf93.qza \
--p-min-samples 2 \
--p-min-frequency 93

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-final-MANGY-ms2-mf93.qza \
--o-visualization BNW-skin-table-final-MANGY-ms2-mf93.qzv

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-final-SEVEREonly.qza \
--o-filtered-table BNW-skin-table-final-SEVERE-ms2-mf158.qza \
--p-min-samples 2 \
--p-min-frequency 158

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-final-SEVERE-ms2-mf158.qza \
--o-visualization BNW-skin-table-final-SEVERE-ms2-mf158.qzv

# Core analysis of filtered 

In [ ]:
qiime feature-table core-features \
--i-table BNW-skin-table-final-OBSHEALTHY-ms2-mf67.qza \
--o-visualization BNW-skin-table-final-OBSHEALTHYonly-ms2-mf67-CORE-FEATURES.qzv

In [ ]:
qiime feature-table core-features \
--i-table BNW-skin-table-final-CONFHEALTHY-ms2-mf198.qza \
--o-visualization BNW-skin-table-final-CONFHEALTHYonly-ms2-mf198-CORE-FEATURES.qzv

In [ ]:
qiime feature-table core-features \
--i-table BNW-skin-table-final-UNCERTAIN-ms2-mf389.qza \
--o-visualization BNW-skin-table-final-UNCERTAINonly-ms2-mf389-CORE-FEATURES.qzv

In [ ]:
qiime feature-table core-features \
--i-table BNW-skin-table-final-MANGY-ms2-mf93.qza \
--o-visualization BNW-skin-table-final-MANGYonly-ms2-mf93-CORE-FEATURES.qzv

In [ ]:
qiime feature-table core-features \
--i-table BNW-skin-table-final-SEVERE-ms2-mf158.qza \
--o-visualization BNW-skin-table-final-SEVEREonly-ms2-mf158-CORE-FEATURES.qzv

# Merge filtred tables together to look at core features across all wombats

In [ ]:
qiime feature-table merge \
--i-tables BNW-skin-table-final-OBSHEALTHY-ms2-mf67.qza BNW-skin-table-final-CONFHEALTHY-ms2-mf198.qza BNW-skin-table-final-UNCERTAIN-ms2-mf389.qza BNW-skin-table-final-SEVERE-ms2-mf158.qza BNW-skin-table-final-MANGY-ms2-mf93.qza \
--o-merged-table BNW-skin-table-final-FILTERED-MERGED.qza

In [ ]:
qiime feature-table core-features \
--i-table BNW-skin-table-final-FILTERED-MERGED.qza \
--o-visualization BNW-skin-table-final-FILTERED-MERGED-CORE-FEATURES.qzv

# Export files to analyse in R

In [ ]:
# export unrooted tree
qiime tools export \
--input-path BNW-skin-sepp-tree.qza \
--output-path /Users/christinanaesborg/Desktop/Microbiome

In [ ]:
# export taxonomy
qiime tools export \
--input-path BNW-skin-silva-132.qza \
--output-path /Users/christinanaesborg/Desktop/Microbiome

In [ ]:
# export filtered table
qiime tools export \
--input-path BNW-table-atleast-12336-reads.qza \
--output-path /Users/christinanaesborg/Desktop/Microbiome

In [ ]:
qiime tools export \
--input-path observed_features.qza \
--output-path /Users/christinanaesborg/Desktop/Microbiome

In [ ]:
# Following script will change the first line of our file to desired header
sed 's/Feature ID/#OTUID/' /Users/christinanaesborg/Desktop/Microbiome/taxonomy.tsv | sed 's/Taxon/taxonomy/' | sed 's/Confidence/confidence/' > /Users/christinanaesborg/Desktop/Microbiome/biom-taxonomy.tsv

In [ ]:
biom add-metadata \
-i /Users/christinanaesborg/Desktop/Microbiome/feature-table.biom \
-o /Users/christinanaesborg/Desktop/Microbiome/table-with-taxonomyv2.biom \
--observation-metadata-fp /Users/christinanaesborg/Desktop/Microbiome/biom-taxonomy.tsv \
--sc-separated taxonomy

In [ ]:
qiime tools export 
--input-path faiths-pd-vector.qza 
--output-path /Users/christinanaesborg/Desktop/Microbiome/QIIME2_outputs/16S

In [ ]:
qiime taxa collapse \
--i-table BNW-skin-reindexed-table.qza \
--i-taxonomy BNW-skin-silva-132.qza \
--p-level 6 \
--output-dir /Biom

qiime feature-table summarize \
--i-table collapsed_table.qza \
--o-visualization collapsed_table.qzv

# GreenGenes database - we ended up using SILVA in the final analysis

In [ ]:
qiime tools import \
  --type 'FeatureData[Sequence]' \
  --input-path /Users/christinanaesborg/Desktop/gg_13_8_otus/rep_set/99_otus.fasta  \                            
  --output-path gg-99_otus.qza

In [ ]:
qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-format HeaderlessTSVTaxonomyFormat \
  --input-path /Users/christinanaesborg/Desktop/gg_13_8_otus/taxonomy/99_otu_taxonomy.txt  \                             
  --output-path gg-99_ref-taxonomy.qz

In [ ]:
qiime vsearch cluster-features-closed-reference \
  --i-table BNW-skin-table.qza \
  --i-sequences BNW-skin-rep-seqs.qza \
  --i-reference-sequences gg-99_otus.qza \
  --p-perc-identity 0.99 \
  --o-clustered-table gg-99_cluster-table.qza \
  --o-unmatched-sequences unmatched.qza \
  --o-clustered-sequences gg-99_rep.qz

In [ ]:
# extract reference reads - primers are 515F/806R
qiime feature-classifier extract-reads \
  --i-sequences gg-99_otus.qza \
  --p-f-primer GTGCCAGCMGCCGCGGTAA \
  --p-r-primer GGACTACHVGGGTWTCTAAT \
  --p-trunc-len 120 \
  --p-min-length 100 \
  --p-max-length 400 \
  --o-reads gg_ref-seqs.qza

In [ ]:
# train naive bayes classifier
qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads gg_ref-seqs.qza \
  --i-reference-taxonomy gg-99_ref-taxonomy.qza \
  --o-classifier gg-99_classifier.qza

In [ ]:
qiime feature-classifier classify-sklearn \
  --i-classifier gg-99_classifier.qza \
  --i-reads BNW-skin-rep-seqs.qza \
  --o-classification BNW-skin-gg_138.qza

In [ ]:
qiime metadata tabulate \
--m-input-file BNW-skin-gg_138.qza \
--o-visualization BNW-skin-gg_138.qzv

In [ ]:
qiime fragment-insertion sepp \
--i-representative-sequences BNW-skin-rep-seqs.qza \
--i-reference-database /Users/christinanaesborg/Desktop/Microbiome_raw\ samples/sepp-refs-gg-13-8.qza  \              
--o-tree BNW-skin-sepp-tree-gg.qza \            
--o-placements BNW-skin-sepp-placements-gg.qza \
--verbose \  
--p-threads 2

In [ ]:
qiime fragment-insertion filter-features \
  --i-table BNW-skin-table.qza \
  --i-tree BNW-skin-sepp-tree-gg.qza \
  --o-filtered-table BNW-skin-filtered_table-gg.qza \
  --o-removed-table BNW-skin-removed_table-gg.qza \
  --verbose

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-removed_table-gg.qza \
--o-visualization BNW-skin-removed_table-gg.qzv

In [ ]:
# alpha rarefaction
qiime diversity alpha-rarefaction \
--i-table BNW-skin-table.qza \
--m-metadata-file /Users/christinanaesborg/Desktop/BNW-skin-metadata.txt   \
--o-visualization BNW-skin-table-rarefaction-gg.qzv \
--p-min-depth 500 \
--p-max-depth 50000

# output: BNW-skin-table-rarefaction.qzv

In [ ]:
# Generate phylogenetic tree for diversity analysis using mafft and fasttree
qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences BNW-skin-rep-seqs.qza \
  --o-alignment gg-aligned-rep-seqs.qza \
  --o-masked-alignment gg-masked-aligned-rep-seqs.qza \
  --o-tree gg-unrooted-tree.qza \
  --o-rooted-tree gg-rooted-tree.qza

In [ ]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny gg-rooted-tree.qza \
  --i-table BNW-skin-table.qza \
  --p-sampling-depth 12105 \
  --m-metadata-file /Users/christinanaesborg/Desktop/BNW-skin-metadata.tsv  \
  --output-dir gg-core-metrics-results

In [ ]:
# alpha diversity
qiime diversity alpha-group-significance \
  --i-alpha-diversity gg-core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file BNW-skin-metadata.txt \
  --o-visualization gg-core-metrics-results/faith-pd-group-significance.qzv

In [ ]:
# alpha diversity
qiime diversity alpha-group-significance \
  --i-alpha-diversity gg-core-metrics-results/evenness_vector.qza \
  --m-metadata-file BNW-skin-metadata.txt \
  --o-visualization gg-core-metrics-results/evenness-group-significance.qzv

In [ ]:
# beta diversity - body site
qiime diversity beta-group-significance \
  --i-distance-matrix gg-core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.txt \
  --m-metadata-column bodysite \
  --o-visualization gg-core-metrics-results/unweighted-unifrac-body-site-significance.qzv \
  --p-pairwise

In [ ]:
# beta diversity - individuals
qiime diversity beta-group-significance \
  --i-distance-matrix gg-core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --m-metadata-column wombatid \
  --o-visualization gg-core-metrics-results/unweighted-unifrac-subject-group-significance.qzv \
  --p-pairwise

In [ ]:
# beta diversity - infection status
qiime diversity beta-group-significance \
  --i-distance-matrix gg-core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.txt \
  --m-metadata-column infectionstatus \
  --o-visualization gg-core-metrics-results/unweighted-unifrac-infection-status-significance.qzv \
  --p-pairwise